## 1. Load Dataset.

In [1]:
from dataloader import load_citation, load_citation_v2, load_citation_v3
import time
start_time = time.time()
dataf = "../data/"
norm_type = "SymNorm_tildeA"
#norm_type = "sym_normalized_A"
original_graph, L, features, labels, idx_train,idx_val, idx_test, data_package = load_citation_v3(dataf,"Cora",norm_type=norm_type,cuda=False, identity_features=False)
name = "Cora"
print(time.time() - start_time)

0.2651383876800537


In [ ]:
from dataloader import load_new_data
import time
start_time = time.time()
dataf = "../data/"
norm_type = "SymNorm_tildeA"
#norm_type = "sym_normalized_A"
original_graph, L, features, labels, idx_train,idx_val, idx_test, graph_info = load_new_data(dataf,"polblogs",norm_type=norm_type,cuda=False, identity_features=True)
name = "polblogs"
print(time.time() - start_time)

In [ ]:
print(features.shape)

In [ ]:
print(graph_info)


In [ ]:
/github/GNNVis/server/data/g2g

## 2. Load and Evaluate Model.

In [ ]:
import time
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from models import tgGAT
from sklearn.metrics import f1_score
from utils import accuracy, set_seed


model_path = "../models/gat_cora_state.pkt"
#model_path = "../models/mlp_cora_state.pkt"
#torch.save(model.state_dict(), model_path)
#args = [1433,16,7,0.5]
#args = [3703,16,6,0.5]
#args = [500, 16, 3, 0.5]
#args = [1222, 16, 2, 0.5]
args = [1433, 1433, 8, 7, 1]

kwargs = {
}
model = tgGAT(*args,**kwargs)
model.load_state_dict(torch.load(model_path, map_location='cpu'))

model.eval()
output = model(data_package)
#loss_test = F.nll_loss(output[data.train_mask],labels[idx_test])
acc_all = accuracy(output,data_package.y)
print(acc_all.item())
#acc_test = accuracy(output[idx_test],labels[idx_test])
#print(loss_test.item(), acc_test.item())



In [ ]:
import time
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from models import GCN
from sklearn.metrics import f1_score
from utils import accuracy, set_seed


model_path = "../models/gcn_photo_state.pkt"
#model_path = "../models/mlp_cora_state.pkt"
#torch.save(model.state_dict(), model_path)
#args = [1433,16,7,0.5]
#args = [3703,16,6,0.5]
#args = [500, 16, 3, 0.5]
#args = [1222, 16, 2, 0.5]
args = [745, 16, 8, 0.5]

kwargs = {
    "bias": True,
}
model = GCN(*args,**kwargs)
model.load_state_dict(torch.load(model_path, map_location='cpu'))

model.eval()
output = model(features,L)
loss_test = F.nll_loss(output[idx_test],labels[idx_test])
acc_test = accuracy(output[idx_test],labels[idx_test])
print(loss_test.item(), acc_test.item())



## 3. Calculate Node Feature Importance.

In [ ]:
features = features.requires_grad_()
output = model(features,L)
N = 645
node_relevance = torch.zeros_like(output)
node_relevance[N] = 1
output.backward(node_relevance)

node_feature_importance = features.grad
node_importance = features.grad.pow(2).sum(dim=1)
node_importance = node_importance.tolist()

#for i in range(len(node_importance)):
#    if not node_importance[i] == 0:
#        pass
#        #print(i, node_importance[i])


In [ ]:
features_importance = features.grad[645]
features_importance = features_importance.tolist()
feature_index = sorted(range(len(features_importance)), key=lambda k: -features_importance[k])
for index in feature_index:
    print(index, features_importance[index])


In [ ]:
graph_out = net(batch)[0]

N = node_no
node_relevance = torch.zeros_like(graph_out.node_features)
node_relevance[N] = 1

graph_in.zero_grad_()
graph_out.node_features.backward(node_relevance)

node_importance = batch.node_features.grad.pow(2).sum(dim=1)
edge_importance = batch.edge_features.grad.pow(2).sum(dim=1)
return node_importance, edge_importance

## 4. Visualize Weights in Model.

In [ ]:
model.state_dict()["gc1.weight"]

In [ ]:
import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set()
#uniform_data = np.random.rand(10, 12)
ax = sns.heatmap(model.state_dict()["gc2.weight"].cpu().numpy())
fig = ax.get_figure()
fig.savefig("gc2_weight.png")

In [ ]:
print(original_graph._indices()[0])

In [ ]:
print(original_graph)

## 5. Visualize Graph.

In [ ]:
import networkx as nx
from fa2 import ForceAtlas2
import matplotlib.pyplot as plt

G = nx.Graph()
indices = original_graph._indices().tolist()
edge_num = len(indices[0])
edge_index = [(indices[0][i], indices[1][i]) for i in range(edge_num)]
node_index = list(range(features.shape[0]))
G.add_nodes_from(node_index)
G.add_edges_from(edge_index)


In [ ]:

forceatlas2 = ForceAtlas2(
                        # Behavior alternatives
                        outboundAttractionDistribution=True,  # Dissuade hubs
                        linLogMode=False,  # NOT IMPLEMENTED
                        adjustSizes=False,  # Prevent overlap (NOT IMPLEMENTED)
                        edgeWeightInfluence=1.0,

                        # Performance
                        jitterTolerance=1.0,  # Tolerance
                        barnesHutOptimize=True,
                        barnesHutTheta=1.2,
                        multiThreaded=False,  # NOT IMPLEMENTED

                        # Tuning
                        scalingRatio=2.0,
                        strongGravityMode=False,
                        gravity=1.0,

                        # Log
                        verbose=True)

positions = forceatlas2.forceatlas2_networkx_layout(G, pos=None, iterations=500)
nx.draw_networkx_nodes(G, positions, node_size=20, with_labels=False, node_color="blue", alpha=0.4)
nx.draw_networkx_edges(G, positions, edge_color="green", alpha=0.05)
plt.axis('off')
plt.show()

# equivalently
#import igraph
#G = igraph.Graph.TupleList(G.edges(), directed=False)
#layout = forceatlas2.forceatlas2_igraph_layout(G, pos=None, iterations=2000)
#igraph.plot(G, layout).show()

In [ ]:
print(positions)

In [ ]:
newlist = list(positions.keys())
newlist.sort()
newPos = []
for i in range(len(newlist)):
    newPos.append([positions[i][0], positions[i][1]])

In [ ]:
len(newlist)

In [ ]:
import pickle as pkl
with open("../data/{}/{}_layout.pkt".format(name,name),"wb") as f:
    pkl.dump(newPos, f)

In [ ]:
import pickle as pkl
with open("../data/{}/{}_layout.pkt".format(name,name),"rb") as f:
    newPos = pkl.load(f)
    print(newPos)

## 6. GCN hook definition.

In [ ]:
import math
import torch
import torch.nn as nn 
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from torch.nn.modules.module import Module
from torch.autograd import Variable
import numpy as np
import skcuda.linalg as sklin
from layer import GCN_layer
from utils import accuracy
'''
GCN_layer(ind,outd,bias=True)
'''

class GCN_hook(nn.Module):
    def __init__(self, num_feature,num_hidden,num_class,dropout,bias=True):
        super(GCN_hook,self).__init__()

        self.gc1 = GCN_layer(num_feature, num_hidden)
        self.gc2 = GCN_layer(num_hidden, num_class)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x1 = F.dropout(x, self.dropout, training=self.training)
        x2 = self.gc2(x1, adj)
        return F.log_softmax(x2, dim=1), x1

In [ ]:
name2 = "photo"

In [ ]:
model_path = "../models/gcn_{}_state.pkt".format(name2)
#torch.save(model.state_dict(), model_path)
#args_input = [3703,16,6,0.5]
#args_input = [500, 16, 3, 0.5]
#args_input = [2879, 16, 7, 0.5]
#args_input = [1222, 16, 2, 0.5]
args_input = [745, 16, 8, 0.5]
kwargs = {
    "bias": True
}
model = GCN_hook(*args_input,**kwargs)
model.load_state_dict(torch.load(model_path, map_location='cpu'))

model.eval()
output, inner_state = model(features,L)
loss_test = F.nll_loss(output[idx_test],labels[idx_test])
acc_test = accuracy(output[idx_test],labels[idx_test])
print(loss_test.item(), acc_test.item())


## 7. T-SNE Visualization for Input, Hidden, Output.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import time
import pickle as pkl

def dimension_reduction(input_array):
    start_time = time.time()
    X = np.array(input_array)    
    X_embedded = TSNE(n_components=2).fit_transform(X)
    print(time.time() - start_time)
    return X_embedded
def visualize(embedded_array,labels):
    
    #sns.set_palette(sns.color_palette("Paired"))
    X = np.array(embedded_array)
    labels = np.array(labels)
    labels = np.expand_dims(labels, axis=1)
    data = np.concatenate((X, labels), axis=1)
    df = pd.DataFrame(data, columns=["x", "y","Labels"])
    # Create an array with the colors you want to use
    colors = ["#FF0B04", "#4374B3"]
    # Set your custom color palette
    customPalette = sns.set_palette(sns.color_palette(colors))
    ax = sns.scatterplot(x="x", y="y",hue="Labels", data=df, palette="Set1", legend=False)
    
current_palette = sns.color_palette()
sns.palplot(current_palette)



In [ ]:
## Input Layer
features_array = features.cpu().detach().numpy()
features_embeddded_array = dimension_reduction(features_array)
visualize(features_embeddded_array, labels.cpu().detach().numpy())
with open("../data/{}/{}_tsne_input.pkt".format(name,name),"wb") as f:
    pkl.dump(features_embeddded_array, f)
    print("Done!")
    

In [ ]:
## Hidden Layer
labels_array = labels.cpu().detach().numpy()
layer2 = inner_state
layer2_array = layer2.cpu().detach().numpy()
layer2_embeddded_array = dimension_reduction(layer2_array)
visualize(layer2_embeddded_array, labels.cpu().detach().numpy())
with open("../data/{}/{}_tsne_hidden.pkt".format(name,name),"wb") as f:
    pkl.dump(layer2_embeddded_array, f)
    print("Done!")

In [ ]:
## Output Layer
input_array = output.cpu().detach().numpy()
labels_array = labels.cpu().detach().numpy()
embedded_array = dimension_reduction(input_array)
visualize(embedded_array, labels.cpu().detach().numpy())
with open("../data/{}/{}_tsne_output.pkt".format(name,name),"wb") as f:
    pkl.dump(embedded_array, f)
    print("Done!")